In [1]:
import numpy as np
import pandas as pd
import h5py
import scipy.sparse as sps
from datetime import timedelta
import datetime

# pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.display.min_rows = 50
pd.options.display.precision = 2
pd.options.display.expand_frame_repr = True


In [2]:
directory = "/workdir/data/mimic-iii/"
filename = directory + 'all_hourly_data.h5'

# from preprocess helpers
lab_var_names = [
 'dbp',
 'fio2',
 'GCS',
 'hr',
 'map',
 'sbp',
 'spontaneousrr',
 'spo2',
 'temp',
# 'urine',
 'bun',
 'magnesium',
 'platelets',
 'sodium',
 'alt',
 'hct',
 'po2',
 'ast',
 'potassium',
 'wbc',
 'bicarbonate',
 'creatinine',
 'lactate',
 'pco2',
 'glucose',
 'inr',
 'hgb',
 'bilirubin_total']


In [3]:
with h5py.File(filename, "r") as f:
    print("Keys: %s" % f.keys())


Keys: <KeysViewHDF5 ['codes', 'interventions', 'patients', 'vitals_labs', 'vitals_labs_mean']>


In [4]:
patients = pd.read_hdf(filename, key="patients")

one_hot = pd.get_dummies(patients[["gender", "admission_type", "first_careunit"]])
patients = patients.join(one_hot)
#display(patients.columns)

# include only patients with 6h < los < 600h or same for los_icu, but los_icu is in days
LOS_LOWER_THRESH = 6 #exclude if LOS is shorter than this many hours
LOS_UPPER_THRESH = 600 #exclude if LOS is longer than this many hours
patients["los"] = (patients["dischtime"] - patients["admittime"])
patients = patients[((patients["los"] > timedelta(hours=LOS_LOWER_THRESH))  & (patients["los"] < timedelta(hours=LOS_UPPER_THRESH))) | 
                    ((patients["los_icu"] * 24 > LOS_LOWER_THRESH)            & (patients["los_icu"] * 24 < LOS_UPPER_THRESH))]

MIN_AGE_THRESH = 18
patients = patients[patients["age"] > MIN_AGE_THRESH]

#fix ages set to 300+ for anonymity purposes
patients.loc[patients['age'] > 90, 'age'] = 90

# ignore following dummies, common practice to exclude 1 dummy per category
# gender_M
# admission_type_ELECTIVE
# first_careunit_CCU

patients = patients[["age", "gender_F", "admission_type_EMERGENCY", "admission_type_URGENT", "first_careunit_CSRU", "first_careunit_MICU", "first_careunit_SICU", "first_careunit_TSICU"]]
patients.columns = ['age', 'is_F', 'is_emergency', 'is_urgent', 'is_csru', 'is_micu', 'is_sicu', 'is_tsicu']
patients.reset_index(inplace=True)
patients.set_index(["subject_id"], inplace=True)
patients.drop(columns=["hadm_id", "icustay_id"], inplace=True)

# normalize
patients = ( patients - patients.mean() ) / patients.std()

print(patients.shape)
display(patients.head())


(37543, 8)


,age,is_F,is_emergency,is_urgent,is_csru,is_micu,is_sicu,is_tsicu
subject_id,,,,,,,,
3,0.72,-0.88,0.48,-0.17,-0.5,1.35,-0.44,-0.39
4,-0.92,1.14,0.48,-0.17,-0.5,1.35,-0.44,-0.39
6,0.12,1.14,-2.09,-0.17,-0.5,-0.74,2.26,-0.39
9,-1.26,-0.88,0.48,-0.17,-0.5,1.35,-0.44,-0.39
11,-0.79,1.14,0.48,-0.17,-0.5,-0.74,2.26,-0.39


In [5]:
interventions = pd.read_hdf(filename, key="interventions")

interventions["vasopressor"] = interventions[['vaso', 'vasopressin']].any(axis='columns').astype(int)
interventions = interventions[["vasopressor"]]
print(interventions.shape)
display(interventions.head())

(3183638, 1)


vasopressor
subject_id hadm_id icustay_id hours_in             
3          145834  211552     0                   0
                              1                   1
                              2                   1
                              3                   1
                              4                   1

In [6]:
# exclude patients that get vasopressor in the first 6h e.g. 0-5
excluded_subject_ids = interventions[interventions.index.get_level_values("hours_in").isin(range(6)) & interventions["vasopressor"] == 1]
excluded_subject_ids = excluded_subject_ids.index.get_level_values("subject_id").unique()
excluded_subject_ids.shape

(8406,)

In [7]:
outcome = pd.DataFrame(interventions.groupby(["subject_id", "hadm_id", "icustay_id"])["vasopressor"].agg("max"))

outcome.reset_index(inplace=True)
outcome.set_index(["subject_id"], inplace=True)
outcome.drop(columns=["hadm_id", "icustay_id"], inplace=True)

outcome.insert(0, "logit", (~outcome["vasopressor"].astype(bool)).astype(int))

print(outcome.shape)
display(outcome.head())


(37543, 2)


,logit,vasopressor
subject_id,,
3,0,1
4,1,0
6,1,0
9,0,1
11,1,0


In [8]:
vitals_labs_mean = pd.read_hdf(filename, key="vitals_labs_mean")

vitals_labs_mean.reset_index(inplace=True)
vitals_labs_mean.set_index(["subject_id", "hours_in"], inplace=True)
vitals_labs_mean.drop(columns=["hadm_id", "icustay_id"], inplace=True)

# Paper name -> df name
# map (mean arterial pressure) -> pulmonary artery pressure mean
# spontaneousrr (spontaneous respiratory rate) -> respiratory rate
# urine (urine output) -> not found! :/
# alt (alanine transaminase) -> alanine aminotransferase
# ast (aspartate aminotransferase) -> asparate aminotransferase
# inr (international normalised ratio) -> prothrombin time inr
mask = vitals_labs_mean.columns.get_level_values("LEVEL2").isin(["diastolic blood pressure", "fraction inspired oxygen", "glascow coma scale total", "heart rate", 
                                                                 "pulmonary artery pressure mean", "systolic blood pressure", "respiratory rate", "oxygen saturation",
                                                                 "temperature", "urine output", "blood urea nitrogen", "magnesium", "platelets", "sodium", "alanine aminotransferase",
                                                                 "hematocrit", "partial pressure of oxygen", "asparate aminotransferase", "potassium", "white blood cell count",
                                                                 "bicarbonate", "creatinine", "lactate", "partial pressure of carbon dioxide", "glucose", "prothrombin time inr",
                                                                 "hemoglobin", "bilirubin"])

# display(vitals_labs_mean.columns[vitals_labs_mean.columns.get_level_values("LEVEL2").str.contains("weight")]) # search column names
vitals_labs_mean = vitals_labs_mean.iloc[:, mask]
vitals_labs_mean = vitals_labs_mean[~vitals_labs_mean.index.get_level_values("subject_id").isin(excluded_subject_ids)]
vitals_labs_mean.columns = lab_var_names

# use only first 6 hours -> can be replaced by last 6h or random sampling, method not indicated in paper
# TODO random time between LOS_LOWER_THRESH hours_in and 90% quantile of LOS
# CONTROL_ENDTIME_UPPERQUANT = 0.9
# endtimes = np.random.uniform(LOS_LOWER_THRESH, CONTROL_ENDTIME_UPPERQUANT * patients['los_icu'])
vitals_labs_mean = vitals_labs_mean[vitals_labs_mean.index.get_level_values("hours_in").isin(range(6))]
vitals_labs_mean_nan = vitals_labs_mean.copy()


median = vitals_labs_mean.median() # median before or after filling?
vitals_labs_mean = vitals_labs_mean.groupby('subject_id').fillna(method='ffill') # ffill by group over hours_in
vitals_labs_mean = vitals_labs_mean.fillna(median) # fill remaining by median

# normalize
vitals_labs_mean = ( vitals_labs_mean - vitals_labs_mean.mean() ) / vitals_labs_mean.std()

print(vitals_labs_mean.shape)
display(vitals_labs_mean.head(10))


/tmp/ipykernel_621240/2546610807.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  vitals_labs_mean.drop(columns=["hadm_id", "icustay_id"], inplace=True)


(174822, 27)


dbp  fio2   GCS    hr   map   sbp  spontaneousrr  spo2  \
subject_id hours_in                                                            
4          0        -0.10 -0.09  0.03 -0.15 -0.30 -0.26      -6.54e-02 -0.15   
           1        -0.10 -0.09  0.03 -0.15 -0.30 -0.26       5.77e-03 -0.15   
           2        -0.10 -0.09  0.03 -0.15 -0.30 -0.26       5.77e-03 -0.15   
           3        -0.10 -0.09  0.03 -0.15 -0.30 -0.26      -4.21e-01 -0.15   
           4        -0.10 -0.09  0.03 -0.15 -0.30 -0.26      -1.36e-01 -0.15   
           5        -0.11 -0.05 -0.70  0.30 -0.72 -0.56      -1.36e-01 -0.15   
6          0        -0.12 -0.10 -2.17 -0.34  2.02  7.73      -4.92e-01 -0.15   
           1        -0.12 -0.10 -2.17 -0.34  2.02  7.73      -6.54e-02 -0.15   
           2        -0.12 -0.10 -2.17 -0.34  2.02  7.73      -6.54e-02 -0.15   
           3        -0.12 -0.10 -2.17 -0.34  2.02  7.73      -3.50e-01 -0.15   

                     temp   bun  magnesium  platelets  sodium   alt   hct  \
subject_id hours_in                                                         
4          0          0.4 -0.17      -0.06      -0.31   -0.49 -0.05 -0.05   
           1          0.4 -0.17       1.16      -0.31   -0.49 -0.05 -0.05   
           2          0.4 -0.17       0.17      -0.31   -0.49 -0.05 -0.05   
           3          0.4 -0.17      -0.29      -0.31   -0.49 -0.05 -0.05   
           4          0.4 -0.17      -0.63      -0.31   -0.49 -0.05 -0.05   
           5          0.4  0.53      -0.17      -0.35   -0.44 -0.05 -0.05   
6          0          0.4  1.07      -0.12      -0.72   -0.82 -0.13  0.27   
           1          0.4  1.07      -0.06      -0.55   -0.82 -0.13  0.27   
           2          0.4  1.07      -0.23      -0.55   -0.82 -0.13  0.27   
           3          0.4  1.07      -0.29      -0.55   -0.82 -0.13  0.27   

                      po2   ast  potassium   wbc  bicarbonate  creatinine  \
subject_id hours_in                                                         
4          0        -0.87 -1.74      -0.06 -0.17        -0.06       -0.14   
           1         0.21 -1.74      -0.06 -0.17        -0.06       -0.14   
           2         0.48 -1.74      -0.06 -0.17        -0.06       -0.14   
           3         0.75 -1.74      -0.06 -0.17        -0.06       -0.14   
           4         0.75 -1.74      -0.06 -0.17        -0.06       -0.14   
           5        -0.60 -1.74      -0.06 -0.31        -1.32       -0.22   
6          0         0.75 -0.05      -0.06  0.78         1.97        0.02   
           1         0.75 -0.05      -0.06  0.78         1.97       -0.22   
           2         0.75 -0.05      -0.06  0.78         1.97       -0.22   
           3         0.75 -0.05      -0.06  0.78         1.97       -0.22   

                     lactate  pco2  glucose   inr   hgb  bilirubin_total  
subject_id hours_in                                                       
4          0           -0.01 -0.05     0.11 -0.08 -0.02            -0.13  
           1           -0.01 -0.05     0.11 -0.32  0.99            -0.13  
           2           -0.01 -0.05     0.11 -0.32  0.99            -0.13  
           3           -0.01 -0.05     0.11 -1.04  0.99            -0.13  
           4           -0.01 -0.05     0.11 -1.04  0.99            -0.13  
           5           -0.01 -0.05     0.54 -1.04  0.99            -0.25  
6          0           -0.01 -0.98    -0.11  0.87 -0.02            -0.14  
           1           -0.01 -0.79    -0.11  0.87 -0.25            -0.14  
           2           -0.01 -1.17    -0.11  1.83 -0.25            -0.14  
           3           -0.01 -0.42    -0.11  0.82 -0.25            -0.14

In [9]:
# # Check stuff
# v2 = vitals_labs_mean.reset_index() # before it is truncated
# maxhours = v2.groupby("subject_id")["hours_in"].max()
# display(maxhours.head())
# print(maxhours.shape)

# res = pd.merge(patients, maxhours, on = "subject_id")
# res["los"] = res["los"].astype(int).astype(float) / 3.6e+12 # convert ns to hours
# res["los_icu"] = (res["los_icu"] * 24).astype(int)
# bool = res["hours_in"].equals(res["los_icu"])
# print(f"Hoursin corresponds to los_icu: {bool}")
# display(res[["los", "los_icu", "hours_in"]])
# # => YES

In [10]:
# FORMAT TO TENSOR (N x V x T)

def convert_2d_to_3d_tensor(data):
    subject_ids = data.index.get_level_values("subject_id").unique()

    N = subject_ids.size
    V = data.shape[1]
    T = 6
    
    # fill 3d array
    tensor = np.zeros((N, T, V))
    for index, row in data.iterrows():
        # display(index)
        # display(row.shape)
        subject_id = index[0]
        hours_in = index[1]
        N_index = subject_ids.get_loc(subject_id)    # map subject_id to index [0, unique length]
        
        tensor[N_index, hours_in, :] = row
    return tensor


# vitals_labs_mean_3d = convert_2d_vitals_labs_to_3d_tensor(vitals_labs_mean)
# vitals_labs_mean_nan_3d = convert_2d_vitals_labs_to_3d_tensor(vitals_labs_mean_nan)
# print(vitals_labs_mean_3d.shape)
# display(vitals_labs_mean_3d[0,:,0])


In [11]:
# display(vitals_labs_mean_nan.head())

indicators = (~vitals_labs_mean_nan.isna()).astype(int)
indicators.columns = [c + "_ind" for c in indicators.columns]
print(indicators.shape)
display(indicators.head())

# indicators_3d = ~np.isnan(vitals_labs_mean_nan_3d)
# print(indicators_3d.shape)
# display(indicators_3d[0,:,0])



(174822, 27)


dbp_ind  fio2_ind  GCS_ind  hr_ind  map_ind  sbp_ind  \
subject_id hours_in                                                         
4          0               0         0        0       0        0        0   
           1               0         0        0       0        0        0   
           2               0         0        0       0        0        0   
           3               0         0        0       0        0        0   
           4               0         0        0       0        0        0   

                     spontaneousrr_ind  spo2_ind  temp_ind  bun_ind  \
subject_id hours_in                                                   
4          0                         0         0         0        0   
           1                         1         0         1        0   
           2                         0         0         0        0   
           3                         1         0         0        0   
           4                         1         0         0        0   

                     magnesium_ind  platelets_ind  sodium_ind  alt_ind  \
subject_id hours_in                                                      
4          0                     0              1           1        1   
           1                     1              0           0        0   
           2                     1              0           0        0   
           3                     1              0           0        0   
           4                     1              0           0        0   

                     hct_ind  po2_ind  ast_ind  potassium_ind  wbc_ind  \
subject_id hours_in                                                      
4          0               0        1        1              0        0   
           1               0        1        0              0        0   
           2               0        1        0              0        0   
           3               0        1        0              0        0   
           4               0        1        0              0        0   

                     bicarbonate_ind  creatinine_ind  lactate_ind  pco2_ind  \
subject_id hours_in                                                           
4          0                       0               0            0         0   
           1                       0               0            0         0   
           2                       0               0            0         0   
           3                       0               0            0         0   
           4                       0               0            0         0   

                     glucose_ind  inr_ind  hgb_ind  bilirubin_total_ind  
subject_id hours_in                                                      
4          0                   0        0        0                    0  
           1                   0        1        1                    0  
           2                   0        0        0                    0  
           3                   0        1        0                    0  
           4                   0        1        0                    0

In [12]:
print(f"Exclude {excluded_subject_ids.size} patients...")

patients = patients[~patients.index.get_level_values("subject_id").isin(excluded_subject_ids)]
print(patients.shape)

outcome = outcome[~outcome.index.get_level_values("subject_id").isin(excluded_subject_ids)]
print(outcome.shape)

# vitals_labs_mean = vitals_labs_mean[~vitals_labs_mean.index.get_level_values("subject_id").isin(excluded_subject_ids)]
print(vitals_labs_mean.shape)

# indicators = indicators[~indicators.index.get_level_values("subject_id").isin(excluded_subject_ids)]
print(indicators.shape)

X = pd.merge(vitals_labs_mean, indicators, how="inner", left_index=True, right_index=True)
X = pd.merge(X, patients, how="inner", left_index=True, right_index=True)
print(X.shape)
display(X.head())

Exclude 8406 patients...
(29137, 8)
(29137, 2)
(174822, 27)
(174822, 27)
(174822, 62)


dbp  fio2   GCS    hr  map   sbp  spontaneousrr  spo2  \
subject_id hours_in                                                          
4          0        -0.1 -0.09  0.03 -0.15 -0.3 -0.26      -6.54e-02 -0.15   
           1        -0.1 -0.09  0.03 -0.15 -0.3 -0.26       5.77e-03 -0.15   
           2        -0.1 -0.09  0.03 -0.15 -0.3 -0.26       5.77e-03 -0.15   
           3        -0.1 -0.09  0.03 -0.15 -0.3 -0.26      -4.21e-01 -0.15   
           4        -0.1 -0.09  0.03 -0.15 -0.3 -0.26      -1.36e-01 -0.15   

                     temp   bun  magnesium  platelets  sodium   alt   hct  \
subject_id hours_in                                                         
4          0          0.4 -0.17      -0.06      -0.31   -0.49 -0.05 -0.05   
           1          0.4 -0.17       1.16      -0.31   -0.49 -0.05 -0.05   
           2          0.4 -0.17       0.17      -0.31   -0.49 -0.05 -0.05   
           3          0.4 -0.17      -0.29      -0.31   -0.49 -0.05 -0.05   
           4          0.4 -0.17      -0.63      -0.31   -0.49 -0.05 -0.05   

                      po2   ast  potassium   wbc  bicarbonate  creatinine  \
subject_id hours_in                                                         
4          0        -0.87 -1.74      -0.06 -0.17        -0.06       -0.14   
           1         0.21 -1.74      -0.06 -0.17        -0.06       -0.14   
           2         0.48 -1.74      -0.06 -0.17        -0.06       -0.14   
           3         0.75 -1.74      -0.06 -0.17        -0.06       -0.14   
           4         0.75 -1.74      -0.06 -0.17        -0.06       -0.14   

                     lactate  pco2  glucose   inr   hgb  bilirubin_total  \
subject_id hours_in                                                        
4          0           -0.01 -0.05     0.11 -0.08 -0.02            -0.13   
           1           -0.01 -0.05     0.11 -0.32  0.99            -0.13   
           2           -0.01 -0.05     0.11 -0.32  0.99            -0.13   
           3           -0.01 -0.05     0.11 -1.04  0.99            -0.13   
           4           -0.01 -0.05     0.11 -1.04  0.99            -0.13   

                     dbp_ind  fio2_ind  GCS_ind  hr_ind  map_ind  sbp_ind  \
subject_id hours_in                                                         
4          0               0         0        0       0        0        0   
           1               0         0        0       0        0        0   
           2               0         0        0       0        0        0   
           3               0         0        0       0        0        0   
           4               0         0        0       0        0        0   

                     spontaneousrr_ind  spo2_ind  temp_ind  bun_ind  \
subject_id hours_in                                                   
4          0                         0         0         0        0   
           1                         1         0         1        0   
           2                         0         0         0        0   
           3                         1         0         0        0   
           4                         1         0         0        0   

                     magnesium_ind  platelets_ind  sodium_ind  alt_ind  \
subject_id hours_in                                                      
4          0                     0              1           1        1   
           1                     1              0           0        0   
           2                     1              0           0        0   
           3                     1              0           0        0   
           4                     1              0           0        0   

                     hct_ind  po2_ind  ast_ind  potassium_ind  wbc_ind  \
subject_id hours_in                                                      
4          0               0        1        1              0        0   
           1               0        1        0              0        0   
        

In [13]:
X3d = convert_2d_to_3d_tensor(X)
print(X3d.shape)
display(X3d[0,:,0])

(29137, 6, 62)


array([-0.09696495, -0.09696495, -0.09696495, -0.09696495, -0.09696495,
       -0.11414187])

In [14]:
# current_time = datetime.datetime.now().strftime("%Y_%m_%d__%H_%M")

OUTPUT_PATH = directory + "vasopressor-"

# X.to_pickle(OUTPUT_PATH + 'Xdata.p')
# outcome.to_pickle(OUTPUT_PATH + 'Ylogits.p')

np.save(OUTPUT_PATH + 'Xdata.npy', X3d)
np.save(OUTPUT_PATH + 'Ylogits.npy', outcome)
np.save(OUTPUT_PATH + 'column_names.npy', X.columns.values)

# LOAD
# patients = pd.read_pickle("./2023_10_19__12_49-vasopressor-patients.npy")
# outcome = pd.read_pickle("./2023_10_19__12_49-vasopressor-outcome.npy")
# vitals_labs_mean = np.load("./2023_10_19__12_49-vasopressor-vitals_labs_mean.npy")
# indicators = np.load("./2023_10_19__12_49-vasopressor-indicators.npy")

print('Dumped')

Dumped
